# Intro to Numba
Numba is a Just-In-Time(JIT) compiler, optimal for high-computation Python. It can work for both CPU and GPU. 

Project Webpage: [https://numba.pydata.org/](https://numba.pydata.org/) \
Numba for GPU: [https://numba.readthedocs.io/en/stable/cuda/index.html](https://numba.readthedocs.io/en/stable/cuda/index.html)

In [ ]:
from numba import cuda
import numpy as np
import cupy as cp

import matplotlib.pyplot as plt
import time

## Numba on the GPU

When using Numba on the GPU, using CuPy arrays is best.

**We have two steps:**
- **Kernel declaration**: 
  - A kernel function is a GPU function that is meant to be called from CPU code.
  - Cannot explicitly return a value
  - Is compiled once, it can be called multiple times with different block sizes or grid sizes)
- **Kernel invocation:**
  - Instantiate the kernel, by a **number of blocks** (or “blocks per grid”), and a **number of threads per block**
  - Passing it the input array to the kernel and running it.
  - Kernels run asynchronously. Use `cuda.synchronize()` to wait for all previous kernel launches to finish executing.

Write just-in-time compiled kernels with Numba using the `@cuda.jit` decorator. 

Access thread position information using the following:
* `cuda.grid(ndim)` returns absolute position of current thread in entire grid of blocks, given `ndim` is the number of dimensions the kernel was instantiated with.
* `cuda.gridSize(ndim)` returns absolute size(shape) of the entire grid of blocks, in units of threads, given `ndim` is the number of dimensions the kernel was instantiated with.

In [ ]:
@cuda.jit
def numba_double(x):
    start = cuda.grid(1)     # absolute position of the current thread in the entire grid, here, ndim=1
    step = cuda.gridsize(1)  # absolute size(shape) of the entire grid of blocks

    for i in range(start, x.shape[0], step):
        x[i] = 2*x[i]

In [ ]:
@cuda.jit
def numba_double_simple(x):
    start = cuda.grid(1)     # absolute position of the current thread in the entire grid, here, ndim=1

    for i in range(start, x.shape[0], 1):
        x[i] = 2*x[i]

When we call the JIT kernel, we have to specify the `[blocks_per_grid, threads_per_block]`.

Compare the performances of CPU NumPy, CPU Numba + NumPy, and GPU Numba + CuPy.

Note: You may get a performance warning due to using a Numba CUDA kernel with Numpy. Since we're comparing performance, ignore the warning and rerun the cell.

In [ ]:
sizes = [2**12, 2**13, 2**14, 2**15, 2**16, 2**17, 2**18, 2**19, 2**20, 2**21]
numpy_times = []
numba_numpy_times = []
numba_cupy_times = []

for size in sizes:
    x_cpu = np.random.randn(size)
    x_gpu = cp.asarray(x_cpu)

    threads_per_block = 32
    blocks_per_grid = (x_cpu.size + threads_per_block - 1) // threads_per_block

    # CPU NumPy time
    start = time.perf_counter()
    output = x_cpu*2
    numpy_times.append(time.perf_counter() - start)

    # CPU Numba + NumPy time
    start = time.perf_counter()
    numba_double[blocks_per_grid, threads_per_block](x_cpu)
    numba_numpy_times.append(time.perf_counter() - start)

    # GPU Numba + CuPy time
    start = time.perf_counter()
    numba_double[blocks_per_grid, threads_per_block](x_gpu)
    numba_cupy_times.append(time.perf_counter() - start)

In [ ]:
plt.xlabel("Problem size")
plt.ylabel("Time to solve")
plt.plot(sizes, numpy_times, label="NumPy")
plt.plot(sizes, numba_numpy_times, label="Numba CPU")
plt.plot(sizes, numba_cupy_times, label="Numba GPU")
plt.legend()
plt.show()

Here is an example of matrix multiplication, made faster by using shared memory.

`cuda.threadIdx.x` gives the thread indices in the current thread block (in the range \[0, `cuda.blockDim`\]).

In [ ]:
# Computation will be done on blocks of threads_per_block x threads_per_block elements.

@cuda.jit
def fast_matmul(A, B, C):
    # Define arrays in the shared memory (size & type must be known at compile time)
    sA = cuda.shared.array(shape=(threads_per_block, threads_per_block), dtype=float64)
    sB = cuda.shared.array(shape=(threads_per_block, threads_per_block), dtype=float64)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    blocks_per_grid = cuda.gridDim.x

    # Quit if (x, y) is outside of valid C boundary
    if x >= C.shape[0] and y >= C.shape[1]:
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of threads_per_block-length vectors.
    tmp = float64(0.)
    for i in range(blocks_per_grid):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * threads_per_block]
        sB[tx, ty] = B[tx + i * threads_per_block, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(threads_per_block):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

## Cleanup

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)